In [1]:
!pwd

/home/kionkim/work/stat-story/Soft_Decision_Tree


In [1]:
import os
import time
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook

In [2]:
args = {}
args['batch_size'] = 64
args['epochs'] = 40
args['input_dim'] = 784
args['lmbda'] = 0.1
args['log_interval'] = 10
args['lr'] = 0.01
args['max_depth'] = 2
args['momentum'] = 0.5
args['no_cuda'] = False
args['cuda'] = True
args['output_dim'] = 10
args['seed'] = 1

In [3]:
class InnerNode():

    def __init__(self, depth, **kwargs):
        self.args = kwargs
        
        self.fc = nn.Linear(self.args['input_dim'], 1)
        beta = torch.randn(1)
        #beta = beta.expand((self.args.batch_size, 1))
        if self.args['cuda']:
            beta = beta.cuda()
        self.beta = nn.Parameter(beta)
        self.leaf = False
        self.prob = None
        self.leaf_accumulator = []
        self.lmbda = self.args['lmbda'] * 2 ** (-depth)
        self.build_child(depth)
        self.penalties = []

    def reset(self):
        self.leaf_accumulator = []
        self.penalties = []
        self.left.reset()
        self.right.reset()

    def build_child(self, depth):
        if depth < self.args['max_depth']:
            self.left = InnerNode(depth+1, **self.args)
            self.right = InnerNode(depth+1, **self.args)
        else :
            self.left = LeafNode(self.args)
            self.right = LeafNode(self.args)

    def forward(self, x):
        return(F.sigmoid(self.beta*self.fc(x)))
    
    def select_next(self, x):
        prob = self.forward(x)
        if prob < 0.5:
            return(self.left, prob)
        else:
            return(self.right, prob)

    def cal_prob(self, x, path_prob):
        self.data = x
        self.prob = self.forward(x) #probability of selecting right node
        self.path_prob = path_prob
        left_leaf_accumulator = self.left.cal_prob(x, path_prob * (1-self.prob))
        right_leaf_accumulator = self.right.cal_prob(x, path_prob * self.prob)
        self.leaf_accumulator.extend(left_leaf_accumulator)
        self.leaf_accumulator.extend(right_leaf_accumulator)
        return(self.leaf_accumulator)

    def get_penalty(self):
        penalty = (torch.sum(self.prob * self.path_prob) / torch.sum(self.path_prob), self.lmbda)
        if not self.left.leaf:
            left_penalty = self.left.get_penalty()
            right_penalty = self.right.get_penalty()
            self.penalties.append(penalty)
            self.penalties.extend(left_penalty)
            self.penalties.extend(right_penalty)
        return(self.penalties)

In [4]:
class LeafNode():
    def __init__(self, args):
        self.args = args
        self.param = torch.randn(self.args['output_dim'])
        if self.args['cuda']:
            self.param = self.param.cuda()
        self.param = nn.Parameter(self.param)
        self.leaf = True
        self.softmax = nn.Softmax()

    def forward(self):
        return(self.softmax(self.param.view(1,-1)))

    def reset(self):
        pass

    def cal_prob(self, x, path_prob):
        self.data = x
        Q = self.forward()
        #Q = Q.expand((self.args.batch_size, self.args.output_dim))
        Q = Q.expand((path_prob.size()[0], self.args['output_dim']))
        return([[path_prob, Q]])

In [5]:
class SoftDecisionTree(nn.Module):

    def __init__(self, **kwargs):
        super(SoftDecisionTree, self).__init__()
        self.args = kwargs
        self.root = InnerNode(1, **self.args)
        self.collect_parameters() ##collect parameters and modules under root node
        self.optimizer = optim.SGD(self.parameters(), lr=self.args['lr'], momentum=self.args['momentum'])
        self.test_acc = []
        self.define_extras(self.args['batch_size'])
        self.best_accuracy = 0.0

    def define_extras(self, batch_size):
        ##define target_onehot and path_prob_init batch size, because these need to be defined according to batch size, which can be differ
        self.target_onehot = torch.FloatTensor(batch_size, self.args['output_dim'])
        self.target_onehot = Variable(self.target_onehot)
        self.path_prob_init = Variable(torch.ones(batch_size, 1))
        if self.args['cuda']:
            self.target_onehot = self.target_onehot.cuda()
            self.path_prob_init = self.path_prob_init.cuda()
    '''
    def forward(self, x):
        node = self.root
        path_prob = Variable(torch.ones(self.args.batch_size, 1))
        while not node.leaf:
            node, prob = node.select_next(x)
            path_prob *= prob
        return node()
    '''        
    def cal_loss(self, x, y):
        batch_size = y.size()[0]
        leaf_accumulator = self.root.cal_prob(x, self.path_prob_init)
        loss = 0.
        max_prob = [-1. for _ in range(batch_size)]
        max_Q = [torch.zeros(self.args['output_dim']) for _ in range(batch_size)]
        for (path_prob, Q) in leaf_accumulator:
            TQ = torch.bmm(y.view(batch_size, 1, self.args['output_dim']), torch.log(Q).view(batch_size, self.args['output_dim'], 1)).view(-1,1)
            loss += path_prob * TQ
            path_prob_numpy = path_prob.cpu().data.numpy().reshape(-1)
            for i in range(batch_size):
                if max_prob[i] < path_prob_numpy[i]:
                    max_prob[i] = path_prob_numpy[i]
                    max_Q[i] = Q[i]
        loss = loss.mean()
        penalties = self.root.get_penalty()
        C = 0.
        for (penalty, lmbda) in penalties:
            C -= lmbda * 0.5 *(torch.log(penalty) + torch.log(1-penalty))
        output = torch.stack(max_Q)
        self.root.reset() ##reset all stacked calculation
        return(-loss + C, output) ## -log(loss) will always output non, because loss is always below zero. I suspect this is the mistake of the paper?

    def collect_parameters(self):
        nodes = [self.root]
        self.module_list = nn.ModuleList()
        self.param_list = nn.ParameterList()
        while nodes:
            node = nodes.pop(0)
            if node.leaf:
                param = node.param
                self.param_list.append(param)
            else:
                fc = node.fc
                beta = node.beta
                nodes.append(node.right)
                nodes.append(node.left)
                self.param_list.append(beta)
                self.module_list.append(fc)

    def train_(self, train_loader, epoch):
        self.train()
        self.define_extras(self.args['batch_size'])
        for batch_idx, (data, target) in enumerate(train_loader):
            correct = 0
            if self.args['cuda']:
                data, target = data.cuda(), target.cuda()
            #data = data.view(self.args.batch_size,-1)
            target = Variable(target)
            target_ = target.view(-1,1)
            batch_size = target_.size()[0]
            data = data.view(batch_size,-1)
            ##convert int target to one-hot vector
            data = Variable(data)
            if not batch_size == self.args['batch_size']: #because we have to initialize parameters for batch_size, tensor not matches with batch size cannot be trained
                self.define_extras(batch_size)
            self.target_onehot.data.zero_()            
            self.target_onehot.scatter_(1, target_, 1.)
            self.optimizer.zero_grad()

            loss, output = self.cal_loss(data, self.target_onehot)
            loss.backward(retain_variables=True)
            self.optimizer.step()
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum()
            accuracy = 100. * correct / len(data)

            if batch_idx % self.args['log_interval'] == 0:
                tqdm.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {}/{} ({:.4f}%)'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0],
                    correct, len(data),
                    accuracy))

    def test_(self, test_loader, epoch):
        self.eval()
        self.define_extras(self.args['batch_size'])
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            if self.args['cuda']:
                data, target = data.cuda(), target.cuda()
            target = Variable(target)
            target_ = target.view(-1,1)
            batch_size = target_.size()[0]
            data = data.view(batch_size,-1)
            ##convert int target to one-hot vector
            data = Variable(data)
            if not batch_size == self.args['batch_size']: #because we have to initialize parameters for batch_size, tensor not matches with batch size cannot be trained
                self.define_extras(batch_size)
            self.target_onehot.data.zero_()            
            self.target_onehot.scatter_(1, target_, 1.)
            _, output = self.cal_loss(data, self.target_onehot)
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum()
        accuracy = 100. * correct / len(test_loader.dataset)
        tqdm.write('\nTest set: Accuracy: {}/{} ({:.4f}%)\n'.format(
            correct, len(test_loader.dataset),
            accuracy))
        self.test_acc.append(accuracy)

        if accuracy > self.best_accuracy:
            self.save_best('./result')
            self.best_accuracy = accuracy

    def save_best(self, path):
        try:
            os.makedirs('./result')
        except:
            print('directory ./result already exists')

        with open(os.path.join(path, 'best_model.pkl'), 'wb') as output_file:
            pickle.dump(self, output_file)

In [6]:
torch.manual_seed(args['seed'])
if args['cuda']:
    torch.cuda.manual_seed(args['seed'])

try:
    os.makedirs('./data')
except:
    print('directory ./data already exists')

directory ./data already exists


In [7]:
import torch
import torch.utils.data
from torchvision import datasets, transforms

kwargs = {'num_workers': 1, 'pin_memory': True} if args['cuda'] else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)


In [8]:
model = SoftDecisionTree(**args)
model

SoftDecisionTree (
  (module_list): ModuleList (
    (0): Linear (784 -> 1)
    (1): Linear (784 -> 1)
    (2): Linear (784 -> 1)
  )
  (param_list): ParameterList (
  )
)

In [125]:
output.data.max(1)

(
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
  0.3449
 [torch.cuda.FloatTensor of size 64 (GPU 0)], 
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
  6
 [torch.cuda.LongTensor of size 64 (GPU 0)])

In [10]:
if args['cuda']:
    model.cuda()

for epoch in tqdm_notebook(range(args['epochs']), desc='epochs'):
    model.train_(train_loader, epoch)
    model.test_(test_loader, epoch)
  
save_result()

A Jupyter Widget

/home/kionkim/anaconda3/envs/kion_env_pytorch/lib/python3.6/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.549688, Accuracy: 5/64 (7.8125%)
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.651123, Accuracy: 4/64 (6.2500%)
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.678881, Accuracy: 9/64 (14.0625%)
Train Epoch: 0 [1920/60000 (3%)]	Loss: 2.565126, Accuracy: 9/64 (14.0625%)
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.529630, Accuracy: 11/64 (17.1875%)
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.465570, Accuracy: 8/64 (12.5000%)
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.500396, Accuracy: 8/64 (12.5000%)
Train Epoch: 0 [4480/60000 (7%)]	Loss: 2.440427, Accuracy: 14/64 (21.8750%)
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.422049, Accuracy: 13/64 (20.3125%)
Train Epoch: 0 [5760/60000 (10%)]	Loss: 2.436325, Accuracy: 15/64 (23.4375%)
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.290283, Accuracy: 15/64 (23.4375%)
Train Epoch: 0 [7040/60000 (12%)]	Loss: 2.285512, Accuracy: 12/64 (18.7500%)
Train Epoch: 0 [7680/60000 (13%)]	Loss: 2.304356, Accuracy: 11/64 (17.1875%)
Train Epoch: 0 [8320

Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.691940, Accuracy: 18/64 (28.1250%)
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.733909, Accuracy: 23/64 (35.9375%)
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.700159, Accuracy: 24/64 (37.5000%)
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.695268, Accuracy: 22/64 (34.3750%)
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.703195, Accuracy: 23/64 (35.9375%)
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.640237, Accuracy: 32/64 (50.0000%)
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.562299, Accuracy: 31/64 (48.4375%)
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.672872, Accuracy: 28/64 (43.7500%)
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.786010, Accuracy: 20/64 (31.2500%)
Train Epoch: 1 [13440/60000 (22%)]	Loss: 1.638038, Accuracy: 23/64 (35.9375%)
Train Epoch: 1 [14080/60000 (23%)]	Loss: 1.681091, Accuracy: 23/64 (35.9375%)
Train Epoch: 1 [14720/60000 (25%)]	Loss: 1.665601, Accuracy: 23/64 (35.9375%)
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.633495, Accuracy: 31/64 (

Train Epoch: 2 [15360/60000 (26%)]	Loss: 1.488822, Accuracy: 21/64 (32.8125%)
Train Epoch: 2 [16000/60000 (27%)]	Loss: 1.525042, Accuracy: 30/64 (46.8750%)
Train Epoch: 2 [16640/60000 (28%)]	Loss: 1.596404, Accuracy: 21/64 (32.8125%)
Train Epoch: 2 [17280/60000 (29%)]	Loss: 1.477512, Accuracy: 24/64 (37.5000%)
Train Epoch: 2 [17920/60000 (30%)]	Loss: 1.663631, Accuracy: 19/64 (29.6875%)
Train Epoch: 2 [18560/60000 (31%)]	Loss: 1.394212, Accuracy: 29/64 (45.3125%)
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.438818, Accuracy: 24/64 (37.5000%)
Train Epoch: 2 [19840/60000 (33%)]	Loss: 1.491806, Accuracy: 27/64 (42.1875%)
Train Epoch: 2 [20480/60000 (34%)]	Loss: 1.545399, Accuracy: 23/64 (35.9375%)
Train Epoch: 2 [21120/60000 (35%)]	Loss: 1.610617, Accuracy: 22/64 (34.3750%)
Train Epoch: 2 [21760/60000 (36%)]	Loss: 1.491325, Accuracy: 24/64 (37.5000%)
Train Epoch: 2 [22400/60000 (37%)]	Loss: 1.475932, Accuracy: 25/64 (39.0625%)
Train Epoch: 2 [23040/60000 (38%)]	Loss: 1.449077, Accuracy: 26/

...

Train Epoch: 35 [39680/60000 (66%)]	Loss: 1.303783, Accuracy: 27/64 (42.1875%)
Train Epoch: 35 [40320/60000 (67%)]	Loss: 1.245115, Accuracy: 30/64 (46.8750%)
Train Epoch: 35 [40960/60000 (68%)]	Loss: 1.358076, Accuracy: 22/64 (34.3750%)
Train Epoch: 35 [41600/60000 (69%)]	Loss: 1.254935, Accuracy: 31/64 (48.4375%)
Train Epoch: 35 [42240/60000 (70%)]	Loss: 1.242547, Accuracy: 30/64 (46.8750%)
Train Epoch: 35 [42880/60000 (71%)]	Loss: 1.138631, Accuracy: 27/64 (42.1875%)
Train Epoch: 35 [43520/60000 (72%)]	Loss: 1.334104, Accuracy: 20/64 (31.2500%)
Train Epoch: 35 [44160/60000 (74%)]	Loss: 1.386184, Accuracy: 23/64 (35.9375%)
Train Epoch: 35 [44800/60000 (75%)]	Loss: 1.262724, Accuracy: 28/64 (43.7500%)
Train Epoch: 35 [45440/60000 (76%)]	Loss: 1.206241, Accuracy: 23/64 (35.9375%)
Train Epoch: 35 [46080/60000 (77%)]	Loss: 1.419565, Accuracy: 23/64 (35.9375%)
Train Epoch: 35 [46720/60000 (78%)]	Loss: 1.310341, Accuracy: 30/64 (46.8750%)
Train Epoch: 35 [47360/60000 (79%)]	Loss: 1.278184, 

Train Epoch: 36 [46720/60000 (78%)]	Loss: 1.402225, Accuracy: 19/64 (29.6875%)
Train Epoch: 36 [47360/60000 (79%)]	Loss: 1.324175, Accuracy: 22/64 (34.3750%)
Train Epoch: 36 [48000/60000 (80%)]	Loss: 1.177114, Accuracy: 25/64 (39.0625%)
Train Epoch: 36 [48640/60000 (81%)]	Loss: 1.274956, Accuracy: 27/64 (42.1875%)
Train Epoch: 36 [49280/60000 (82%)]	Loss: 1.326806, Accuracy: 23/64 (35.9375%)
Train Epoch: 36 [49920/60000 (83%)]	Loss: 1.251319, Accuracy: 28/64 (43.7500%)
Train Epoch: 36 [50560/60000 (84%)]	Loss: 1.301559, Accuracy: 24/64 (37.5000%)
Train Epoch: 36 [51200/60000 (85%)]	Loss: 1.123936, Accuracy: 29/64 (45.3125%)
Train Epoch: 36 [51840/60000 (86%)]	Loss: 1.415264, Accuracy: 23/64 (35.9375%)
Train Epoch: 36 [52480/60000 (87%)]	Loss: 1.302889, Accuracy: 22/64 (34.3750%)
Train Epoch: 36 [53120/60000 (88%)]	Loss: 1.321573, Accuracy: 29/64 (45.3125%)
Train Epoch: 36 [53760/60000 (90%)]	Loss: 1.265938, Accuracy: 28/64 (43.7500%)
Train Epoch: 36 [54400/60000 (91%)]	Loss: 1.336141, 

Train Epoch: 37 [53120/60000 (88%)]	Loss: 1.409808, Accuracy: 22/64 (34.3750%)
Train Epoch: 37 [53760/60000 (90%)]	Loss: 1.147467, Accuracy: 27/64 (42.1875%)
Train Epoch: 37 [54400/60000 (91%)]	Loss: 1.155540, Accuracy: 29/64 (45.3125%)
Train Epoch: 37 [55040/60000 (92%)]	Loss: 1.220857, Accuracy: 25/64 (39.0625%)
Train Epoch: 37 [55680/60000 (93%)]	Loss: 1.367723, Accuracy: 30/64 (46.8750%)
Train Epoch: 37 [56320/60000 (94%)]	Loss: 1.433375, Accuracy: 16/64 (25.0000%)
Train Epoch: 37 [56960/60000 (95%)]	Loss: 1.318302, Accuracy: 24/64 (37.5000%)
Train Epoch: 37 [57600/60000 (96%)]	Loss: 1.466608, Accuracy: 23/64 (35.9375%)
Train Epoch: 37 [58240/60000 (97%)]	Loss: 1.220073, Accuracy: 26/64 (40.6250%)
Train Epoch: 37 [58880/60000 (98%)]	Loss: 1.406736, Accuracy: 19/64 (29.6875%)
Train Epoch: 37 [59520/60000 (99%)]	Loss: 1.422901, Accuracy: 26/64 (40.6250%)

Test set: Accuracy: 3894/10000 (38.9400%)

Train Epoch: 38 [0/60000 (0%)]	Loss: 1.215291, Accuracy: 27/64 (42.1875%)
Train Epoch: 


Test set: Accuracy: 3891/10000 (38.9100%)

Train Epoch: 39 [0/60000 (0%)]	Loss: 1.229178, Accuracy: 32/64 (50.0000%)
Train Epoch: 39 [640/60000 (1%)]	Loss: 1.303855, Accuracy: 24/64 (37.5000%)
Train Epoch: 39 [1280/60000 (2%)]	Loss: 1.086349, Accuracy: 27/64 (42.1875%)
Train Epoch: 39 [1920/60000 (3%)]	Loss: 1.121485, Accuracy: 35/64 (54.6875%)
Train Epoch: 39 [2560/60000 (4%)]	Loss: 1.363343, Accuracy: 27/64 (42.1875%)
Train Epoch: 39 [3200/60000 (5%)]	Loss: 1.409376, Accuracy: 22/64 (34.3750%)
Train Epoch: 39 [3840/60000 (6%)]	Loss: 1.292175, Accuracy: 25/64 (39.0625%)
Train Epoch: 39 [4480/60000 (7%)]	Loss: 1.308870, Accuracy: 26/64 (40.6250%)
Train Epoch: 39 [5120/60000 (9%)]	Loss: 1.456277, Accuracy: 29/64 (45.3125%)
Train Epoch: 39 [5760/60000 (10%)]	Loss: 1.299022, Accuracy: 28/64 (43.7500%)
Train Epoch: 39 [6400/60000 (11%)]	Loss: 1.238797, Accuracy: 28/64 (43.7500%)
Train Epoch: 39 [7040/60000 (12%)]	Loss: 1.179374, Accuracy: 25/64 (39.0625%)
Train Epoch: 39 [7680/60000 (13%)

NameError: name 'save_result' is not defined

#### Examine

In [15]:
# data 가지고 오기
for i, d in enumerate(train_loader):
    dt = d
    if i ==1:
        break

In [16]:
data = dt[0]
target = dt[1]
data = data.cuda()

In [17]:
# Check shape
dt[0].shape #torch.Size([64, 1, 28, 28])
type(dt[0]) # torch.FloatTensor

dt[1].shape # torch.Size([64])
type(dt[1]) #torch.LongTensor

torch.LongTensor

In [18]:
target = Variable(target) # transform into Variable
target.size() #target.shape이 먹히지 않음... torch.Size([64])
type(target) #torch.autograd.variable.Variable

torch.autograd.variable.Variable

In [19]:
target_ = target.view(-1,1) # 2차원에 dimension을 하나 더함
target_.size() #torch.Size([64, 1])

torch.Size([64, 1])

In [20]:
type(target_)

torch.autograd.variable.Variable

In [21]:
batch_size = target_.size()[0] # 64
target_onehot = torch.FloatTensor(batch_size, args['output_dim'])
# NOTE: 평균 torch.mean, 분산 torch.var
target_onehot.shape #torch.Size([64, 10])

target_onehot = Variable(target_onehot)
target_onehot.size() #torch.Size([64, 10])

# Manually zero the gradients after running the backward pass
target_onehot.data.zero_()
# Cuda variable과 CPU variable 구분이 중요
target_onehot.scatter_(1, target_, 1.)
target = target.cuda()

In [22]:
data = data.view(batch_size, -1)
data.size() # data.shape과 같은 결과... torch.Size([64, 784])
data = Variable(data)
data.size() #torch.Size([64, 784])

torch.Size([64, 784])

In [23]:
data #[torch.cuda.FloatTensor of size 64x784 (GPU 0)]
target #[torch.cuda.LongTensor of size 64 (GPU 0)]

Variable containing:
 3
 2
 9
 8
 8
 4
 2
 6
 0
 9
 0
 2
 5
 8
 2
 6
 4
 2
 9
 2
 3
 3
 4
 3
 0
 1
 7
 6
 3
 3
 8
 0
 5
 5
 4
 3
 0
 8
 6
 4
 1
 1
 9
 1
 2
 1
 1
 9
 3
 8
 2
 5
 2
 3
 9
 5
 4
 2
 3
 9
 5
 8
 5
 3
[torch.cuda.LongTensor of size 64 (GPU 0)]

In [61]:
x =data
y = target_onehot
y = y.cuda()

In [102]:
batch_size = y.size()[0]
leaf_accumulator = model.root.cal_prob(x, model.path_prob_init)
loss = 0.
max_prob = [-1. for _ in range(batch_size)]
max_Q = [torch.zeros(args['output_dim']) for _ in range(batch_size)]

In [81]:
Q = leaf_accumulator[0][1]

In [117]:
Q

Variable containing:
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.

In [104]:
path_prob_numpy

array([ 0.1324345 ,  0.21761641,  0.196484  ,  0.03086069,  0.1792316 ,
        0.12814821,  0.06910293,  0.06686758,  0.13109507,  0.05495748,
        0.270551  ,  0.15169108,  0.11298319,  0.089144  ,  0.2516799 ,
        0.22766237,  0.19082129,  0.20981511,  0.24587752,  0.33108386,
        0.08872376,  0.03934519,  0.06537192,  0.1690933 ,  0.11628345,
        0.24548066,  0.29202336,  0.06440445,  0.14188954,  0.1003928 ,
        0.1047959 ,  0.07431797,  0.11409755,  0.09502421,  0.10942503,
        0.1919713 ,  0.07240174,  0.11217692,  0.16305202,  0.17078111,
        0.32150209,  0.12721629,  0.09401006,  0.40673721,  0.14356177,
        0.31448159,  0.25075132,  0.15921967,  0.17536296,  0.04047649,
        0.11600545,  0.16481802,  0.23023495,  0.12552118,  0.20719177,
        0.10737529,  0.22184153,  0.11038724,  0.23680636,  0.23403534,
        0.16647011,  0.11629023,  0.36176962,  0.1995648 ], dtype=float32)

In [103]:
for i in range(batch_size):
        print('i = {} max_prob = {}'.format(i, max_prob[i]))
        print('i = {} path_prob_numpy = {}'.format(i, path_prob_numpy[i]))
        print('i = {} Q = {}'.format(i, Q[i]))
        print('i = {} max_Q = {}'.format(i, max_Q[i]))
        if max_prob[i] < path_prob_numpy[i]:
            max_prob[i] = path_prob_numpy[i]
            max_Q[i] = Q[i]
            

i = 0 max_prob = -1.0
i = 0 path_prob_numpy = 0.13243450224399567
i = 0 Q = Variable containing:
 0.0078
 0.1170
 0.0253
 0.1556
 0.3831
 0.0326
 0.0824
 0.0438
 0.0583
 0.0940
[torch.cuda.FloatTensor of size 10 (GPU 0)]

i = 0 max_Q = 
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 10]

i = 1 max_prob = -1.0
i = 1 path_prob_numpy = 0.21761640906333923
i = 1 Q = Variable containing:
 0.0078
 0.1170
 0.0253
 0.1556
 0.3831
 0.0326
 0.0824
 0.0438
 0.0583
 0.0940
[torch.cuda.FloatTensor of size 10 (GPU 0)]

i = 1 max_Q = 
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 10]

i = 2 max_prob = -1.0
i = 2 path_prob_numpy = 0.19648399949073792
i = 2 Q = Variable containing:
 0.0078
 0.1170
 0.0253
 0.1556
 0.3831
 0.0326
 0.0824
 0.0438
 0.0583
 0.0940
[torch.cuda.FloatTensor of size 10 (GPU 0)]

i = 2 max_Q = 
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 10]

i = 3 max_prob = -1.0
i = 3 path_prob_numpy = 0.03086068667471409
i = 3 Q = Variable containing:
 0.007

In [116]:
torch.stack(max_Q)

Variable containing:
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.0253  0.1556  0.3831  0.0326  0.0824  0.0438  0.0583  0.0940
 0.0078  0.1170  0.

In [113]:
model

SoftDecisionTree (
  (module_list): ModuleList (
    (0): Linear (784 -> 1)
    (1): Linear (784 -> 1)
    (2): Linear (784 -> 1)
  )
  (param_list): ParameterList (
  )
)

In [88]:
loss = 0.
for (path_prob, Q) in leaf_accumulator:
    TQ = torch.bmm(y.view(batch_size, 1, args['output_dim']), torch.log(Q).view(batch_size, args['output_dim'], 1)).view(-1,1)
    loss += path_prob * TQ
    path_prob_numpy = path_prob.cpu().data.numpy().reshape(-1)
    for i in range(batch_size):
        if max_prob[i] < path_prob_numpy[i]:
            max_prob[i] = path_prob_numpy[i]
            max_Q[i] = Q[i]
    loss = loss.mean()
    penalties = model.root.get_penalty()
    C = 0.
    for (penalty, lmbda) in penalties:
        C -= lmbda * 0.5 *(torch.log(penalty) + torch.log(1-penalty))
    output = torch.stack(max_Q)

RuntimeError: invalid argument 3: sizes do not match at /opt/conda/conda-bld/pytorch_1503970438496/work/torch/lib/THC/generated/../generic/THCTensorMathPointwise.cu:217

In [24]:
model = SoftDecisionTree(**args)
model.cuda()
model.cal_loss(data, target)

RuntimeError: invalid argument 2: size '[64 x 1 x 10]' is invalid for input of with 64 elements at /opt/conda/conda-bld/pytorch_1503970438496/work/torch/lib/TH/THStorage.c:41

In [57]:
path_prob_init = Variable(torch.ones(64, 1))
path_prob_init = path_prob_init.cuda()
root = InnerNode(1, **args)
root.cuda()
leaf_accumulator = root.cal_prob(data, path_prob_init)

AttributeError: 'InnerNode' object has no attribute 'cuda'